# SENG 474
# Assignment 1 - Problem 2
# Nolan Kurylo
# V00893175

To execute notebook, ensure ALL cells are run from top to bottom (since imports/df creation are only called once)


References:

1) https://pandas.pydata.org/pandas-docs/stable/reference/index.html

2) https://www.python-course.eu/Decision_Trees.php

3) https://medium.com/@lope.ai/decision-trees-from-scratch-using-id3-python-coding-it-up-6b79e3458de4

4) https://stackoverflow.com/




In [2]:
import pandas as pd
import numpy as np
import pprint
from matplotlib import pyplot as plt

np.random.seed(1337)

df = pd.read_csv('elections_clean.csv') # create dataframe of csv


label_vector = df.pop('Democrat') # this is the target/label vector
df['Democrat'] = label_vector # place label vector at the end of df
feature_list = ['Education','Religion','EthnicMale','EthnicFemale'] # these will be the features in the tree


def entropy(col):
    """ Finds the entropy of the input column col
    :param col: input column
    :return: the entropy of the column (information gain)
    """
    values, num_values = np.unique(col, return_counts=True) 
    entropy = 0
    for i in range(len(values)): # calculation from lecture slides
        entropy = entropy + (-num_values[i]/np.sum(num_values))*np.log2(num_values[i]/np.sum(num_values))
    return entropy

def conditional_entropy(dataset, col):
    """ Finds the conditional entropy of the subset dataset for the input column col
    :param dataset: input dataframe
    :param col: column of dataframe to find conditional entropy of
    :return: the conditional entropy of the column (information gain)
    """
    values, num_values = np.unique(col, return_counts=True) 
    cond_entropy = 0
    for i in range(len(values)):
        matching_rows = dataset.loc[col==values[i]] # In the column col, find all rows with matching value values[i] 
        label_vector_subset = matching_rows['Democrat'] # return a subset of the values in the label vector
        cond_entropy = cond_entropy + ((num_values[i]/np.sum(num_values)) * entropy(label_vector_subset) ) 
    return cond_entropy

def information_gain(dataset, col): 
    """ Finds the information gain of the subset dataset for the input column col
    :param dataset: input dataframe
    :param col: input column
    :return: the total entropy of the column (information gain)
    """
    total_entropy = entropy(dataset['Democrat']) # get the entropy for the entire label vector column
    cond_entropy = conditional_entropy(dataset, col)
    information_gain = total_entropy - cond_entropy # calc info gain
    return information_gain
   

def max_info_gain(dataset, feature_list): 
    """ Finds the feature still in the feature_list with the maximum information gain
    :param dataset: input dataframe
    :param feature_list: subset of features from the features in the dataset 
    :return feature with the highest information gain
    """
    info_gains = {}

    for feature in feature_list:
        info_gain = information_gain(dataset, dataset[feature])
        info_gains[feature] = info_gain

    return max(info_gains, key=info_gains.get)


def ID3(dataset, feature_list, parent=None):
    """ Recursively builds a dictionary-structured Decision Tree Classifier based on ID3 algorithm
    :param dataset: training set
    :param feature_list: list of all features from the dataset
    :param parent: most common element in the current label vector subset
    :return: Decision Tree structured as a dictionary
    """

    if len(np.unique(dataset["Democrat"])) == 1: # if the label vector is "pure" (only one value in label vector)
        return dataset["Democrat"][0] # return the pure value and go back up tree

    if(len(feature_list) == 0): # reached the end of the branch
        return parent # return parent value and go back up tree

   
    curr_node = max_info_gain(dataset, feature_list) # find the feature with the max information gain and make it the curr_node in the tree

    curr_node_values, num_values = np.unique(dataset[curr_node], return_counts=True) # get the values and their counts associated with the curr_node
    
    parent = dataset['Democrat'].mode()[0] # make parent the most common element in the current label vector subset
    

    DecisionTree = {curr_node:{}}

    feature_list = list(filter(lambda feature: feature != curr_node, feature_list)) # remove the curr_node (.remove() was causing issues)
    

    for value in curr_node_values: #alue associated with the curr_node feature
        
        sub_dataset = dataset[dataset[curr_node] == value].reset_index(drop=True) # split data into a subset that excludes all rows of the current 'value' of the curr_node in the loop
        DecisionTree[curr_node][value] = ID3(sub_dataset, feature_list, parent) # recursively build the tree on the new subset as the dataset
           
    return DecisionTree
        

def split_training_validation_sets(df):
    """ Find 70% of the original dataset as the training set, 30% as the validaiton set
    :param df: dataframe to be split
    :return: training and validation splits as dataframes
    """
    shuffled_dataset = df.sample(frac=1).reset_index(drop=True) # shuffle the dataset
    
    split_70_30 = int(df.shape[0] * 0.7) # find the index for the 70 / 30 split to split on
    training_set = shuffled_dataset.iloc[:split_70_30].reset_index(drop=True) # 70% of dataset
    validation_set = shuffled_dataset.iloc[split_70_30:].reset_index(drop=True) # 30% of dataset
    return training_set,validation_set
    

def predict(validation_sample, DecisionTree): 
    """ Recursively runs through DecisionTree and returns where the input validation_sample evaluates to a 1 or 0 for its label
    :param validation_sample: sample for the Decision tree to predict
    :param DecisionTree: dictionary structured model
    :return: the binary prediction - 0 or 1
    """
    # decsion is made based on the already-built input DecisionTree 
    
    for feature in list(validation_sample.keys()): # for each feature in the sample
        
        if feature not in list(DecisionTree.keys()): #ignore missing features
            continue
        try:
            prediction = DecisionTree[feature][validation_sample[feature]] # see if node in tree is a leaf( 0 or 1) or a dict (sub tree)
            
            
            if(prediction == 1 or prediction == 0): # node is a leaf, return the label value
                
                return prediction
            else: # node is a subtree, recursively go down to next node in subtree
                
                return predict(validation_sample, prediction)

        except:
            return df['Democrat'].mean() # return "Majority vote"
        


def validate(validation_set, DecisionTree):
    """ Finds the accuracy of the DecsionTree given an input validaetion_set and the DecisionTree
    :param validation_set: samples for the Decision tree to predict
    :param DecisionTree: dictionary structured model
    :return: validation accuracy (%)
    """
    
    validation_samples = validation_set.iloc[:,:-1].to_dict(orient='records') # convert dataset to dictionary
    
    actual_output = validation_set.iloc[:,-1] # remove the label vector so we can see how how well the tree predicts the samples
    num_correct = 0 # store the number of samples in the validation_set that are actually correct
    
    for i in range(len(validation_samples)):
        prediciton = predict(validation_samples[i],DecisionTree) # label vector either predicted as 1 or 0 based on DecisionTree
        
        if(prediciton == actual_output[i]): 
            num_correct += 1 # DecisionTree predicted the sample correctly
    
    return (num_correct/len(validation_samples)) * 100.0 # return overall accuracy percentage


def num_stumps(DecisionTree,  stumps = {} ): 
    """ Recursively finds the stumps in the tree and the number of times each of them appear (features and a feature's values can be stumps)
    :param DecisionTree: dictionary structured model
    :param stumps: dict to record number of times each stump has been seen
    :return: stumps dictionary
    """
    for feature in list(DecisionTree.keys()):
        stump = DecisionTree[feature] # see if stump in tree is a leaf( 0 or 1) or a dict (sub tree)
        
        if(stump != 1 and stump != 0): # if its a subtree, increment the number of times this stump has appeared
            if(feature in stumps): # stump has been seen before
                stumps[feature] += 1
            else: # stump has never been seen before
                stumps[feature] = 1
            
            num_stumps(stump, stumps) # recurse down subtree to the next stump
        
    return stumps

def depth(DecisionTree, prev_depth, stumps = {} ):
    """ Recursively fidns the maximum depth of the DecisionTree (not including leaf nodes)
    :param DecisionTree: dictionary structured model
    :param prev_depth: subtree's last seen depth
    :param stumps: dict to record number of times each stump has been seen
    :return: the largest depth of the Decision Tree
    """
    curr_depth = prev_depth # set the current depth to the subtree's last known depth
    for feature in list(DecisionTree.keys()):
        stump = DecisionTree[feature] # see if stump in tree is a leaf( 0 or 1) or a dict (sub tree)
        
        if(stump != 1 and stump != 0):# if stump is a subtree, keep recursing down to find the max depth
            new_depth = depth(stump, prev_depth + 1, stumps) # record the depth of the subtree
            curr_depth = max(new_depth, curr_depth) # store the larger depth of the two

    return curr_depth
        

# Split dataset into training and validation
training_set, validation_set = split_training_validation_sets(df)

# Create the decision tree based on the training set
DecisionTree = ID3(training_set, feature_list) 
print("The DecisionTree's structure:")
pprint.pprint(DecisionTree) # print the dictionary structure of the tree

# Obtain the accuracy/errors of the DecisionTree
training_acc = validate(training_set, DecisionTree)
training_err = 100 - training_acc
print("The training accuracy of the DecisionTree: " + str(training_acc) + "%")
print("The training error of the DecisionTree: " + str(training_err) + "%")
print()

validation_acc = validate(validation_set, DecisionTree)
validation_err = 100 - validation_acc
print("The validation accuracy of the DecisionTree: " + str(validation_acc) + "%")
print("The validation error of the DecisionTree: " + str(validation_err) + "%")
print()

# Report the number of features (also each feature's values) that are repeated in decision stumps
stumps = num_stumps(DecisionTree)
print("The stumps in the DecisionTree and the number of times they are repeated are:")
for stump in stumps:
    print(stump + ": " + str(stumps[stump]))
print()


# Report the maximum depth of the decision tree (depth of root = 1)
max_depth = depth(DecisionTree, 1)
print("Maximum Depth of DecisionTree is: " + str(max_depth))



The DecisionTree's structure:
{'Education': {'High School Diploma Only': {'EthnicFemale': {'Black female': {'EthnicMale': {'Black male': {'Religion': {'Catholic': 1,
                                                                                                                         'Christian Generic': 1}},
                                                                                             'White male': {'Religion': {'Catholic': 1,
                                                                                                                         'Christian Generic': 1}}}},
                                                             'Native American female': {'Religion': {'Catholic': {'EthnicMale': {'Native American male': 1}},
                                                                                                     'Christian Generic': 0}},
                                                             'White female': {'Religion': {'Catholic': {'EthnicMale':